In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

ee.Initialize(project='ee-arzaaan789')

In [2]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("Lepus europaeus.csv", delimiter='\t')
df = df[df["occurrenceStatus"] == "PRESENT"]
df = df[df['year']>=2022]
# df['eventDate'] = df['eventDate'].str.replace('/','')
df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')
df = df[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
df = df.dropna().reset_index(drop=True)

/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_84512/3337580572.py:4: DtypeWarning: Columns (10,39,41,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Lepus europaeus.csv", delimiter='\t')


In [3]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

# ee.Initialize()

# Your dataframe 'df' must have columns: decimalLongitude, decimalLatitude
# Example: df = pd.read_csv("Erinaceus europaeus.csv", delimiter='\t')

def create_aoi(lon, lat, box_size_km=1):
    """Create approx 1km x 1km square polygon around lon, lat."""
    half_side_deg = box_size_km / 111.32 / 2  # Rough approx degrees per km

    coords = [
        [lon - half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat - half_side_deg],
        [lon + half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat + half_side_deg],
        [lon - half_side_deg, lat - half_side_deg]
    ]
    return ee.Geometry.Polygon(coords)

def compute_all_indices(feature, start_date, end_date):
    """Compute spectral indices and LST for one EE Feature (with AOI geometry)."""
    aoi = feature.geometry()

    s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
          .filterBounds(aoi)
          .filterDate(start_date, end_date)
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())
    s2 = ee.Image(s2).clip(aoi)

    blue = s2.select('B2')
    green = s2.select('B3')
    red = s2.select('B4')
    nir = s2.select('B8')
    swir = s2.select('B11')

    L = 0.5  # SAVI constant

    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    savi = nir.subtract(red).divide(nir.add(red).add(L)).multiply(1 + L).rename('SAVI')
    mndwi = green.subtract(swir).divide(green.add(swir)).rename('MNDWI')
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    bsi = (red.add(blue).subtract(nir.add(swir))).divide(red.add(blue).add(nir).add(swir)).rename('BSI')
    ui = nir.subtract(swir).divide(nir.add(swir)).rename('UI')

    reducers = ee.Reducer.mean()
    scale_10m = 1000

    ndvi_mean = ndvi.reduceRegion(reducers, aoi, scale_10m).get('NDVI')
    ndwi_mean = ndwi.reduceRegion(reducers, aoi, scale_10m).get('NDWI')
    ndbi_mean = ndbi.reduceRegion(reducers, aoi, scale_10m).get('NDBI')
    savi_mean = savi.reduceRegion(reducers, aoi, scale_10m).get('SAVI')
    mndwi_mean = mndwi.reduceRegion(reducers, aoi, scale_10m).get('MNDWI')
    ndsi_mean = ndsi.reduceRegion(reducers, aoi, scale_10m).get('NDSI')
    bsi_mean = bsi.reduceRegion(reducers, aoi, scale_10m).get('BSI')
    ui_mean = ui.reduceRegion(reducers, aoi, scale_10m).get('UI')

    # MODIS LST dataset
    modis = (ee.ImageCollection("MODIS/061/MOD11A1")
             .filterBounds(aoi)
             .filterDate(start_date, end_date)
             .select('LST_Day_1km'))

    lst_mean_img = modis.mean().multiply(0.02).clip(aoi)

    lst_mean = lst_mean_img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=1000
    ).get('LST_Day_1km')

    return feature.set({
        'NDVI': ndvi_mean,
        'NDWI': ndwi_mean,
        'NDBI': ndbi_mean,
        'SAVI': savi_mean,
        'MNDWI': mndwi_mean,
        'NDSI': ndsi_mean,
        'BSI': bsi_mean,
        'UI': ui_mean,
        'LST': lst_mean
    })

# Split df into batches
batch_size = 50
batches = [df.iloc[i:i+batch_size] for i in range(0, len(df), batch_size)]

results_list = []

index_names = ['NDVI', 'NDWI', 'NDBI', 'SAVI', 'MNDWI', 'NDSI', 'BSI', 'UI', 'LST']

for batch in tqdm(batches, desc="Processing batches"):
    features = []
    batch_indices = []
    feature_metadata = {}

    for idx, row in batch.iterrows():
        
        aoi = create_aoi(row['decimalLongitude'], row['decimalLatitude'])
        feature = ee.Feature(aoi).set('index', idx)
        features.append(feature)
        batch_indices.append(idx)  # Save the original index
        
        event_date = row['eventDate']
        start_date = (event_date - timedelta(days=15)).strftime('%Y-%m-%d')
        end_date = (event_date + timedelta(days=15)).strftime('%Y-%m-%d')
        feature_metadata[idx] = (start_date, end_date)

    # Create a FeatureCollection from the list of features
    fc = ee.FeatureCollection(features)
    
    # Define wrapper for map to inject per-feature dates
    def map_with_dates(f):
        idx = f.get('index')
        # Use dictionary lookup to get dates for this feature
        date_dict = ee.Dictionary(ee.Dictionary(feature_metadata))
        dates = ee.List(date_dict.get(ee.Number(idx)))
        return compute_all_indices(f, dates.get(0), dates.get(1))

    try:
        result_fc = fc.map(map_with_dates)
        results = result_fc.getInfo()

        rows = []
        for f in results['features']:
            props = f['properties']
            rows.append(props)

    except Exception as e:
        print(f"Error processing batch starting at index {batch.index[0]}: {e}")
        # If there's an error, create placeholder rows with None
        rows = [{'index': i, **{name: None for name in index_names}} for i in batch_indices]

    batch_results_df = pd.DataFrame(rows).sort_values('index').reset_index(drop=True)
    results_list.append(batch_results_df)

# Concatenate all batches and sort by original index
all_results_df = pd.concat(results_list).sort_values('index').reset_index(drop=True)

# Merge with original df
df_final = pd.concat([df.reset_index(drop=True), all_results_df.drop(columns=['index'])], axis=1)

print(df_final.head())


Processing batches: 100%|██████████| 204/204 [19:37<00:00,  5.77s/it]

           species  decimalLatitude  decimalLongitude  eventDate       BSI  \
0  Lepus europaeus        52.746722         -1.038527 2022-10-09 -0.699341   
1  Lepus europaeus        52.819095         -1.096287 2022-07-20 -0.654526   
2  Lepus europaeus        52.583645         -0.894504 2022-05-10 -0.491253   
3  Lepus europaeus        52.723961         -0.565204 2023-05-01 -0.444113   
4  Lepus europaeus        52.709478         -0.891327 2023-05-07 -0.431772   

          LST     MNDWI      NDBI      NDSI      NDVI      NDWI      SAVI  \
0  286.446075 -0.528530 -0.189280 -0.528530  0.684339 -0.650132  1.026362   
1  303.545778 -0.560384 -0.106128 -0.560384  0.603091 -0.628835  0.904530   
2  292.681717 -0.169483 -0.423198 -0.169483  0.629200 -0.553601  0.943726   
3  288.931470 -0.165867 -0.363490 -0.165867  0.547637 -0.499577  0.821378   
4  289.501750 -0.191123 -0.337865 -0.191123  0.546665 -0.486093  0.819932   

         UI  
0  0.189280  
1  0.106128  
2  0.423198  
3  0.363490 

In [4]:
df_final.to_csv('brown_hare_full_data.csv', index=False)
# len(df)
df = pd.read_csv("brown_hare_full_data.csv")

df['eventDate'] = pd.to_datetime(df['eventDate'], format='%Y-%m-%d', errors='coerce')
df = df.dropna().reset_index(drop=True)

In [5]:
winter_wheat = pd.read_csv("winter wheat.csv", delimiter='\t')
winter_wheat = winter_wheat[winter_wheat["occurrenceStatus"] == "PRESENT"]
winter_wheat = winter_wheat[winter_wheat['year'] >= 2022]
winter_wheat['eventDate'] = pd.to_datetime(winter_wheat['eventDate'], format='%Y-%m-%d', errors='coerce')
winter_wheat = winter_wheat[['species', 'decimalLatitude', 'decimalLongitude', 'eventDate']]
winter_wheat = winter_wheat.dropna().reset_index(drop=True)


/var/folders/0n/sggn66691kd6g5mtyzpq2hv80000gn/T/ipykernel_84512/2925844888.py:1: DtypeWarning: Columns (39,41,46) have mixed types. Specify dtype option on import or set low_memory=False.
  winter_wheat = pd.read_csv("winter wheat.csv", delimiter='\t')


In [6]:
import numpy as np
from sklearn.neighbors import BallTree

# Convert lat/lon to radians for BallTree
brown_hare_coords = np.deg2rad(df[['decimalLatitude', 'decimalLongitude']].values)
winter_wheat_coords = np.deg2rad(winter_wheat[['decimalLatitude', 'decimalLongitude']].values)

# Build BallTree using haversine metric
tree = BallTree(winter_wheat_coords, metric='haversine')

# Define search radius: 1 km in radians
earth_radius_km = 6371.0
radius = 1 / earth_radius_km  # 1 km in radians

# Initialize presence columns
df['winter_wheat_presence'] = 0

# Iterate through each brown hare point with tqdm for progress tracking
for i in tqdm(range(len(df))):
    point = brown_hare_coords[i].reshape(1, -1)
    event_month = df.loc[i, 'eventDate'].month
    event_year = df.loc[i, 'eventDate'].year

    # WINTER WHEAT
    idxs = tree.query_radius(point, r=radius)[0]
    for j in idxs:
        winter_wheat_date = winter_wheat.loc[j, 'eventDate']
        if winter_wheat_date.month == event_month and winter_wheat_date.year == event_year:
            df.at[i, 'winter_wheat_presence'] = 1
            break  # Found at least one match, no need to check further

100%|██████████| 10087/10087 [00:01<00:00, 9741.87it/s]


In [7]:
from pyproj import Transformer
from rasterio.windows import Window
import rasterio
from collections import Counter
import numpy as np


land_cover_map = {
    1: "Deciduous woodland",
    2: "Coniferous woodland",
    3: "Arable",
    4: "Improved grassland",
    5: "Neutral grassland",
    6: "Calcareous grassland",
    7: "Acid grassland",
    8: "Fen",
    9: "Heather",
    10: "Heather grassland",
    11: "Bog",
    12: "Inland rock",
    13: "Saltwater",
    14: "Freshwater",
    15: "Supralittoral rock",
    16: "Supralittoral sediment",
    17: "Littoral rock",
    18: "Littoral sediment",
    19: "Saltmarsh",
    20: "Urban",
    21: "Suburban"
}

# Batch coordinate transformation
transformer_ni = Transformer.from_crs("EPSG:4326", "EPSG:29903", always_xy=True)
transformer_gb = Transformer.from_crs("EPSG:4326", "EPSG:27700", always_xy=True)

coords = list(zip(df['decimalLongitude'], df['decimalLatitude']))
df['easting_ni'], df['northing_ni'] = zip(*transformer_ni.itransform(coords))
df['easting_gb'], df['northing_gb'] = zip(*transformer_gb.itransform(coords))

# Raster processing optimization
gb_raster = 'gblcm2023_10m.tif'
n_ireland_raster = 'nilcm2023_10m.tif'


def get_land_cover_class_1km(row):
    try:
        window_size_pixels = 100  # 1 km / 10m resolution

        # Open GB raster and read 1km x 1km window
        with rasterio.open(gb_raster) as src:
            row_idx, col_idx = src.index(row['easting_gb'], row['northing_gb'])

            # Calculate window bounds, making sure not to go out of raster bounds
            row_start = max(row_idx - window_size_pixels // 2, 0)
            col_start = max(col_idx - window_size_pixels // 2, 0)

            # Adjust window size if near edges
            height = min(window_size_pixels, src.height - row_start)
            width = min(window_size_pixels, src.width - col_start)

            window = Window(col_start, row_start, width, height)
            data = src.read(1, window=window)

            # If all zero (or no data), fallback to NI raster
            if np.all(data == 0):
                with rasterio.open(n_ireland_raster) as src_ni:
                    row_idx_ni, col_idx_ni = src_ni.index(row['easting_ni'], row['northing_ni'])

                    row_start_ni = max(row_idx_ni - window_size_pixels // 2, 0)
                    col_start_ni = max(col_idx_ni - window_size_pixels // 2, 0)

                    height_ni = min(window_size_pixels, src_ni.height - row_start_ni)
                    width_ni = min(window_size_pixels, src_ni.width - col_start_ni)

                    window_ni = Window(col_start_ni, row_start_ni, width_ni, height_ni)
                    data_ni = src_ni.read(1, window=window_ni)

                    data = data_ni

            # Find the most common class (mode) ignoring zeros (assuming 0 means no data)
            unique, counts = np.unique(data[data != 0], return_counts=True)
            if len(counts) == 0:
                return "Unknown"

            mode_class = unique[np.argmax(counts)]
            return land_cover_map.get(mode_class, "Unknown")

    except Exception as e:
        print(f"Error processing row: {e}")
        return "Unknown"


tqdm.pandas()
df['Land_cover'] = df.progress_apply(get_land_cover_class_1km, axis=1)


100%|██████████| 10087/10087 [04:15<00:00, 39.47it/s]


In [8]:
df = df.dropna()
df=df.drop(['easting_ni', 'northing_ni', 'easting_gb', 'northing_gb'],axis=1)
# Remove where Land_cover is Unknown
df = df[df['Land_cover'] != "Unknown"].reset_index(drop=True)
df.to_csv("brown_hare_final_data.csv", index=False)
df

,species,decimalLatitude,decimalLongitude,eventDate,BSI,LST,MNDWI,NDBI,NDSI,NDVI,NDWI,SAVI,UI,winter_wheat_presence,Land_cover
0,Lepus europaeus,52.746722,-1.038527,2022-10-09,-0.699341,286.446075,-0.528530,-0.189280,-0.528530,0.684339,-0.650132,1.026362,0.189280,0,Arable
1,Lepus europaeus,52.819095,-1.096287,2022-07-20,-0.654526,303.545778,-0.560384,-0.106128,-0.560384,0.603091,-0.628835,0.904530,0.106128,0,Improved grassland
2,Lepus europaeus,52.583645,-0.894504,2022-05-10,-0.491253,292.681717,-0.169483,-0.423198,-0.169483,0.629200,-0.553601,0.943726,0.423198,0,Improved grassland
3,Lepus europaeus,52.723961,-0.565204,2023-05-01,-0.444113,288.931470,-0.165867,-0.363490,-0.165867,0.547637,-0.499577,0.821378,0.363490,0,Arable
4,Lepus europaeus,52.709478,-0.891327,2023-05-07,-0.431772,289.501750,-0.191123,-0.337865,-0.191123,0.546665,-0.486093,0.819932,0.337865,0,Improved grassland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10082,Lepus europaeus,55.267030,-1.624645,2022-02-07,-0.583027,278.794319,-0.526633,-0.107525,-0.526633,0.526273,-0.599162,0.789296,0.107525,0,Arable
10083,Lepus europaeus,54.977467,-1.919587,2024-05-11,-0.645504,292.649449,-0.417604,-0.296257,-0.417604,0.696344,-0.634863,1.044399,0.296257,0,Arable
10084,Lepus europaeus,55.272401,-1.618299,2022-02-07,-0.516168,278.878166,-0.459324,-0.098333,-0.459324,0.467266,-0.529902,0.700772,0.098333,0,Deciduous woodland
10085,Lepus europaeus,55.261713,-1.649871,2023-02-09,-0.636645,278.456060,-0.421279,-0.249452,-0.421279,0.644174,-0.605173,0.966133,0.249452,0,Arable
